# Upload csv to the database

The data was taken from the link: https://www.kaggle.com/c/home-credit-default-risk/

Task: Upload Excel table to MS SQL Server and combine two of them ('application_test', 'application_train') with deleting one of the columns ('TARGET')

Problem: MS SQL Server cannot correctly transfer data from an Excel table to the expected database if a separator character is present in the column values. As a result, when loading data, the table will be curved.

Below is my version of solving this problem and completing the task with detailed comments.

In [1]:
#Importing required libraries
import pypyodbc 
import numpy as np
import time

In [2]:
#Create a list with table names and connect to the database
tablenames = ['application_test', 'application_train', 'bureau', 'bureau_balance', 'credit_card_balance', 
              'installments_payments', 'POS_CASH_balance', 'previous_application']

connection = pypyodbc.connect('Driver={SQL Server}; SERVER="server"; DATABASE="database"; UID="username"; PWD="password"')

cursor = connection.cursor()

I decided to fix the problem by changing the values themselves.
In my case, there were "," in the table cells. I decided to replace them with "/" so that when loading the tables there would be no problems with the delimiter.

In [3]:
# For each of the tables, we first create a copy with the corrections already made 
# After that we send the SQL-request for creating tables.

for tablename in tablenames:
    
    # Started copying
    print(tablename, time.ctime())
    fin = open('path/to/' + tablename +'.csv', 'r', encoding='utf-8')
    fout = open('path/to/new/' + tablename + '_edited.csv', 'w', encoding='utf-8')
    s = fin.readline()
    print(s, end='', file=fout) # Migrated column headers
    
    # The correction process has begun
    while s:
        s = fin.readline()
        if not s:
            break
        i_first = s.find('"')
        i_second = s.find('"', i_first + 1)
        while i_first != -1 and i_second != -1:
            st_wrong = s[i_first:i_second + 1]
            st_right = st_wrong.replace(', ', ' / ')
            s = s.replace(st_wrong, st_right[1:-1])
            i_first = s.find('"', i_second + 1)
            i_second = s.find('"', i_first + 1)
        print(s, end='', file=fout)
    fin.close()
    fout.close()
    print(tablename, ' edit is finished', time.ctime()) # Copy and repair completed
    
    # Now create tables in the database
    fin = open('path/to/' + tablename +'_edited.csv', 'r', encoding='utf-8')
    names_of_column = fin.readline().strip().split(',')
    fin.close()
    SQLQuery = ("create table " + tablename + "_edited(" + "\n")
    for name in names_of_column:
        SQLQuery += "\t" + name + " varchar(50)," + "\n"
    SQLQuery += ")"
    cursor.execute(SQLQuery)
    connection.commit()
    print(tablename,  'is created', time.ctime())
    SQLQuery = (
        "bulk insert " + tablename + "_edited\n" +
        "from 'path/to/" + tablename + "_edited.csv'" + "\n" +
        "with (fieldterminator = ',', firstrow = 2)"
    )
    cursor.execute(SQLQuery)
    connection.commit()
    print('Data imported in', tablename, time.ctime())
    
# Change the names of the tables in the list to new ones
for i in range(len(tablenames)):
    tablenames[i] += '_edited'

application_test Tue Jun 16 19:19:05 2020
application_test  edit is finished Tue Jun 16 19:19:05 2020
application_test is created Tue Jun 16 19:19:05 2020
Data imported in application_test Tue Jun 16 19:19:07 2020
application_train Tue Jun 16 19:19:07 2020
application_train  edit is finished Tue Jun 16 19:19:08 2020
application_train is created Tue Jun 16 19:19:08 2020
Data imported in application_train Tue Jun 16 19:19:20 2020
bureau Tue Jun 16 19:19:20 2020
bureau  edit is finished Tue Jun 16 19:19:24 2020
bureau is created Tue Jun 16 19:19:24 2020
Data imported in bureau Tue Jun 16 19:19:39 2020
bureau_balance Tue Jun 16 19:19:39 2020
bureau_balance  edit is finished Tue Jun 16 19:20:22 2020
bureau_balance is created Tue Jun 16 19:20:22 2020
Data imported in bureau_balance Tue Jun 16 19:21:51 2020
credit_card_balance Tue Jun 16 19:21:51 2020
credit_card_balance  edit is finished Tue Jun 16 19:22:00 2020
credit_card_balance is created Tue Jun 16 19:22:00 2020
Data imported in credit_

In [4]:
# We join tables 'application_test' and 'application_train' and delete the column 'TARGET'
cursor.execute("alter table " + tablenames[1] + "\n" + "drop column TARGET;")
connection.commit()
cursor.execute("insert into " + tablenames[0] + "\n" + "select * from " + tablenames[1] + ";")
connection.commit()
cursor.execute("drop table " + tablenames[1] + ";")
connection.commit()
tablenames.pop(1)

'application_train_edited'

Now you need to change the data type of the columns in each table.
Since the set of columns is different everywhere, you need to process each table individually.

In [5]:
# Change the data type in the table 'application_test'
tablename = tablenames[0]

# Let's create a dictionary with a union of columns for the desired data type
tablechange = {
    'FlagYN':['FLAG_OWN_CAR', 'FLAG_OWN_REALTY'], 
    'On_Float':[
        'SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 
        'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 
        'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 
        'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 
        'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 
        'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 
        'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 
        'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 
        'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 
        'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 
        'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 
        'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 
        'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 
        'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 
        'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 
        'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'
    ],
    'Bit':[
        'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'FLAG_DOCUMENT_2', 
        'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 
        'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 
        'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
        'FLAG_DOCUMENT_21'
    ]
}

In [6]:
# Next, change the data types of the specified columns to the desired
for column in tablechange['FlagYN']:
    print(column, end='')
    SQLQuery =("update " + tablename + " set " + column + " = 1\n" + "where " + column + " = 'Y';" + "\n"
              "update " + tablename + " set " + column + " = 0\n" + "where " + column + " = 'N';" + "\n"
              "alter table " + tablename + "\n" + "alter column " + column + " bit;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
for column in tablechange['On_Float']:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
for column in tablechange['Bit']:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " bit;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

FLAG_OWN_CAR is changed Tue Jun 16 19:26:07 2020
FLAG_OWN_REALTY is changed Tue Jun 16 19:26:09 2020
SK_ID_CURR is changed Tue Jun 16 19:26:14 2020
CNT_CHILDREN is changed Tue Jun 16 19:26:17 2020
AMT_INCOME_TOTAL is changed Tue Jun 16 19:26:21 2020
AMT_CREDIT is changed Tue Jun 16 19:26:24 2020
AMT_ANNUITY is changed Tue Jun 16 19:26:26 2020
AMT_GOODS_PRICE is changed Tue Jun 16 19:26:32 2020
REGION_POPULATION_RELATIVE is changed Tue Jun 16 19:26:34 2020
DAYS_BIRTH is changed Tue Jun 16 19:26:40 2020
DAYS_EMPLOYED is changed Tue Jun 16 19:26:42 2020
DAYS_REGISTRATION is changed Tue Jun 16 19:26:48 2020
DAYS_ID_PUBLISH is changed Tue Jun 16 19:26:50 2020
OWN_CAR_AGE is changed Tue Jun 16 19:26:56 2020
CNT_FAM_MEMBERS is changed Tue Jun 16 19:26:58 2020
REGION_RATING_CLIENT is changed Tue Jun 16 19:27:04 2020
REGION_RATING_CLIENT_W_CITY is changed Tue Jun 16 19:27:06 2020
HOUR_APPR_PROCESS_START is changed Tue Jun 16 19:27:12 2020
REG_REGION_NOT_LIVE_REGION is changed Tue Jun 16 19:27:1

In [7]:
# Then repeat the steps for the remaining tables.

# Change the data type in the table 'bureau'
tablename = tablenames[1]
columns = [
    'SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 
    'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 
    'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY'
]
for column in columns:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

SK_ID_CURR is changed Tue Jun 16 19:38:48 2020
SK_ID_BUREAU is changed Tue Jun 16 19:39:11 2020
DAYS_CREDIT is changed Tue Jun 16 19:39:34 2020
CREDIT_DAY_OVERDUE is changed Tue Jun 16 19:39:54 2020
DAYS_CREDIT_ENDDATE is changed Tue Jun 16 19:40:18 2020
DAYS_ENDDATE_FACT is changed Tue Jun 16 19:40:42 2020
AMT_CREDIT_MAX_OVERDUE is changed Tue Jun 16 19:41:06 2020
CNT_CREDIT_PROLONG is changed Tue Jun 16 19:41:31 2020
AMT_CREDIT_SUM is changed Tue Jun 16 19:41:56 2020
AMT_CREDIT_SUM_DEBT is changed Tue Jun 16 19:42:21 2020
AMT_CREDIT_SUM_LIMIT is changed Tue Jun 16 19:42:46 2020
AMT_CREDIT_SUM_OVERDUE is changed Tue Jun 16 19:43:11 2020
DAYS_CREDIT_UPDATE is changed Tue Jun 16 19:43:35 2020
AMT_ANNUITY is changed Tue Jun 16 19:44:00 2020
Table bureau_edited is changed


In [8]:
# Change the data type in the table 'bureau_balance'
tablename = tablenames[2]
columns = ['SK_ID_BUREAU', 'MONTHS_BALANCE']
for column in columns:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

SK_ID_BUREAU is changed Tue Jun 16 19:52:04 2020
MONTHS_BALANCE is changed Tue Jun 16 19:59:52 2020
Table bureau_balance_edited is changed


In [9]:
# Change the data type in the table 'credit_card_balance'
tablename = tablenames[3]
columns = [
    'SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 
    'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 
    'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 
    'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 
    'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF'
]
for column in columns:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

SK_ID_PREV is changed Tue Jun 16 20:00:29 2020
SK_ID_CURR is changed Tue Jun 16 20:01:11 2020
MONTHS_BALANCE is changed Tue Jun 16 20:01:55 2020
AMT_BALANCE is changed Tue Jun 16 20:02:37 2020
AMT_CREDIT_LIMIT_ACTUAL is changed Tue Jun 16 20:03:19 2020
AMT_DRAWINGS_ATM_CURRENT is changed Tue Jun 16 20:04:01 2020
AMT_DRAWINGS_CURRENT is changed Tue Jun 16 20:04:48 2020
AMT_DRAWINGS_OTHER_CURRENT is changed Tue Jun 16 20:05:30 2020
AMT_DRAWINGS_POS_CURRENT is changed Tue Jun 16 20:06:18 2020
AMT_INST_MIN_REGULARITY is changed Tue Jun 16 20:07:02 2020
AMT_PAYMENT_CURRENT is changed Tue Jun 16 20:07:51 2020
AMT_PAYMENT_TOTAL_CURRENT is changed Tue Jun 16 20:08:39 2020
AMT_RECEIVABLE_PRINCIPAL is changed Tue Jun 16 20:09:27 2020
AMT_RECIVABLE is changed Tue Jun 16 20:10:15 2020
AMT_TOTAL_RECEIVABLE is changed Tue Jun 16 20:10:59 2020
CNT_DRAWINGS_ATM_CURRENT is changed Tue Jun 16 20:11:47 2020
CNT_DRAWINGS_CURRENT is changed Tue Jun 16 20:12:35 2020
CNT_DRAWINGS_OTHER_CURRENT is changed Tue

In [10]:
# Change the data type in the table 'installments_payments'
tablename = tablenames[4]
columns = [
    'SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 
    'AMT_INSTALMENT', 'AMT_PAYMENT'
]
for column in columns:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

SK_ID_PREV is changed Tue Jun 16 20:19:01 2020
SK_ID_CURR is changed Tue Jun 16 20:21:43 2020
NUM_INSTALMENT_VERSION is changed Tue Jun 16 20:24:34 2020
NUM_INSTALMENT_NUMBER is changed Tue Jun 16 20:27:24 2020
DAYS_INSTALMENT is changed Tue Jun 16 20:30:24 2020
DAYS_ENTRY_PAYMENT is changed Tue Jun 16 20:33:33 2020
AMT_INSTALMENT is changed Tue Jun 16 20:38:25 2020
AMT_PAYMENT is changed Tue Jun 16 20:48:07 2020
Table installments_payments_edited is changed


In [11]:
# Change the data type in the table 'POS_CASH_balance'
tablename = tablenames[5]
columns = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']
for column in columns:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')

SK_ID_PREV is changed Tue Jun 16 20:50:07 2020
SK_ID_CURR is changed Tue Jun 16 20:52:18 2020
MONTHS_BALANCE is changed Tue Jun 16 20:54:37 2020
CNT_INSTALMENT is changed Tue Jun 16 20:56:58 2020
CNT_INSTALMENT_FUTURE is changed Tue Jun 16 20:59:22 2020
SK_DPD is changed Tue Jun 16 21:01:51 2020
SK_DPD_DEF is changed Tue Jun 16 21:04:29 2020
Table POS_CASH_balance_edited is changed


In [12]:
# Change the data type in the table 'previous_application'
tablename = tablenames[6]
tablechange = {
    'FlagYN':['FLAG_LAST_APPL_PER_CONTRACT'], 
    'On_Float':[
        'SK_ID_PREV', 'SK_ID_CURR', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 
        'HOUR_APPR_PROCESS_START', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 
        'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE',
        'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'
    ],
    'Bit':['NFLAG_LAST_APPL_IN_DAY']
}
for column in tablechange['FlagYN']:
    print(column, end='')
    SQLQuery =("update " + tablename + " set " + column + " = 1\n" + "where " + column + " = 'Y';" + "\n"
              "update " + tablename + " set " + column + " = 0\n" + "where " + column + " = 'N';" + "\n"
              "alter table " + tablename + "\n" + "alter column " + column + " bit;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
for column in tablechange['On_Float']:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " float;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
for column in tablechange['Bit']:
    print(column, end='')
    SQLQuery =("alter table " + tablename + "\n" + "alter column " + column + " bit;")
    cursor.execute(SQLQuery)
    connection.commit()
    print(' is changed', time.ctime())
    
print('Table', tablename, 'is changed')
connection.close()

FLAG_LAST_APPL_PER_CONTRACT is changed Tue Jun 16 21:05:02 2020
SK_ID_PREV is changed Tue Jun 16 21:05:17 2020
SK_ID_CURR is changed Tue Jun 16 21:05:31 2020
AMT_ANNUITY is changed Tue Jun 16 21:05:47 2020
AMT_APPLICATION is changed Tue Jun 16 21:05:59 2020
AMT_CREDIT is changed Tue Jun 16 21:06:15 2020
AMT_DOWN_PAYMENT is changed Tue Jun 16 21:06:31 2020
AMT_GOODS_PRICE is changed Tue Jun 16 21:06:48 2020
HOUR_APPR_PROCESS_START is changed Tue Jun 16 21:07:04 2020
RATE_DOWN_PAYMENT is changed Tue Jun 16 21:07:20 2020
RATE_INTEREST_PRIMARY is changed Tue Jun 16 21:07:36 2020
RATE_INTEREST_PRIVILEGED is changed Tue Jun 16 21:07:53 2020
DAYS_DECISION is changed Tue Jun 16 21:08:16 2020
SELLERPLACE_AREA is changed Tue Jun 16 21:08:35 2020
CNT_PAYMENT is changed Tue Jun 16 21:08:57 2020
DAYS_FIRST_DRAWING is changed Tue Jun 16 21:09:21 2020
DAYS_FIRST_DUE is changed Tue Jun 16 21:09:57 2020
DAYS_LAST_DUE_1ST_VERSION is changed Tue Jun 16 21:10:36 2020
DAYS_LAST_DUE is changed Tue Jun 16 21

As a result, we get correctly loaded tables